In [6]:
import os
import collections
import json
import glob
import re
from src.wimbd_ import _load_dataset
from datasets import load_from_disk

HF_HOME = "/share/edc/home/antonis/datasets/huggingface"
save_path = os.makedirs(os.path.join(HF_HOME, 'wmt09_gens'), exist_ok=True)

base_pth = "/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/models/experiment_6_logits_max_4/"
doc_res_files = glob.glob(os.path.join(base_pth, "**/doc_results.json"), recursive=True)


In [2]:
pattern = re.compile(r'wmt\d{2}-(\w{2}-\w{2})')
language_pairs = [pattern.search(path).group(1) for path in doc_res_files if pattern.search(path)]
doc = json.load(open(doc_res_files[0]))

def find_lang(path):
    pattern = re.compile(r'wmt\d{2}-(\w{2}-\w{2})')
    return pattern.search(path).group(1)

wmt = _load_dataset('wmt')

Loading dataset wmt
Loading dataset for language pair ('cs', 'en')
Loading dataset for language pair ('de', 'en')
Loading dataset for language pair ('fr', 'en')
Loading dataset for language pair ('es', 'en')
Loading dataset for language pair ('it', 'en')
Loading dataset for language pair ('hu', 'en')
Loading dataset for language pair ('en', 'cs')
Loading dataset for language pair ('en', 'de')
Loading dataset for language pair ('en', 'fr')
Loading dataset for language pair ('en', 'es')
Loading dataset for language pair ('en', 'it')
Loading dataset for language pair ('en', 'hu')


In [30]:
from tqdm import tqdm
from datasets import DatasetDict
from datasets import Dataset

def insert_gens_into_wmt(wmt, doc_res_files):
    wmt_model_langs = collections.defaultdict(dict)
    for path in tqdm(doc_res_files, desc="Processing files"):
        wmt_model = wmt.copy()

        with open(path, 'r') as f:
            doc_res = json.load(f)

        model_name = path.split('/')[-5]
        task = path.split('/')[-3]
        lang = find_lang(path)
        lang1, lang2 = lang.split('-')
        total_rows = len(wmt_model[lang]['translation'])
        matched_rows = 0
        doc_res_task_dict = wmt_model[lang].to_dict()
        len_task_dict = len(doc_res_task_dict['translation'])
        unmatched_rows = [i for i in range(len_task_dict)]

        for doc in tqdm(doc_res[task], desc=f"Inserting generations for {task}", leave=False):
            src = doc['src']
            ref = doc['ref']
            gen = doc['result'][0]

            for idx in unmatched_rows:
                row = doc_res_task_dict['translation'][idx]
                # print(row)
                if row[lang1] != src and row[lang2] != ref:
                    continue
                else:
                    doc_res_task_dict['translation'][idx]['gen'] = gen
                    matched_rows += 1
                    unmatched_rows.remove(idx)
                    break
        
        if len(unmatched_rows) > 0:
            print(f"Could not match {len(unmatched_rows)} rows for {task}")
            raise Exception

        wmt_model_langs[model_name][lang] = Dataset.from_dict(doc_res_task_dict)
        print(f"{model_name} {lang} keys: {doc_res_task_dict['translation'][0].keys()}")

    for model_name, model_data in wmt_model_langs.items():
        print(f"{model_name}: {model_data['en-es']['translation'][0].keys()}")
        save_pth = os.path.join(HF_HOME, f'wmt09_gens_{model_name}')
        dataset_dict = DatasetDict(model_data)
        dataset_dict.save_to_disk(save_pth)
        print(f"Saved to {save_pth}")

    return wmt_model_langs


wmt_model = insert_gens_into_wmt(wmt, doc_res_files)

# dataset_dict = {lang: Dataset.from_dict({'translation': value['translation']}) for lang, value in wmt.items()}
# dataset_dict = DatasetDict(dataset_dict)

# Now you can save it to disk
# dataset_dict.save_to_disk(os.path.join(HF_HOME, 'wmt09_gens'))

Processing files:   1%|          | 1/120 [00:00<01:32,  1.28it/s]

OLMo-7B en-hu keys: dict_keys(['en', 'hu', 'gen'])


Processing files:   2%|▏         | 2/120 [00:01<01:09,  1.69it/s]

OLMo-7B de-en keys: dict_keys(['de', 'en', 'gen'])


Processing files:   2%|▎         | 3/120 [00:01<01:03,  1.85it/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/3027 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3027 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3027 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3027 [00:00<?, ? examples/s]

Saved to /share/edc/home/antonis/datasets/huggingface/wmt09_gens_pythia-160m


In [31]:
wmt_model_pth = "/share/edc/home/antonis/datasets/huggingface/wmt09_gens_pythia-160m"
wmt_model = load_from_disk(wmt_model_pth)
wmt_model['en-es']['translation'][1].keys()

dict_keys(['en', 'es', 'gen'])